<a href="https://colab.research.google.com/github/Hari-priya-Sridharan/PagesToPixels/blob/main/PromptToImage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.4/227.4 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.2 MB/s eta 0:00:00


In [ ]:
pip install git+https://github.com/huggingface/diffusers.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 8.4 MB/s eta 0:00:00


In [ ]:
pip install transformers accelerate safetensors

DALL-E-2 Model

In [ ]:
# imports
from openai import OpenAI  # OpenAI Python library to make API calls
import configparser
import requests  # used to download images
import os  # used to access filepaths
from datetime import datetime

# Read API key from credential.ini
config = configparser.ConfigParser()
config.read('credential.ini')
API_KEY = config['openai']['APIKEY']
client = OpenAI(api_key=API_KEY)


def read_file_and_process(filename):
    path = create_directory()

    with open(filename, 'r') as file:
        prompts = file.read()

    prompt = prompts.split('\n\n')
    for i, prompt in enumerate(prompt):
        generation_response = generate_dalle_img(prompt)
        save_image(generation_response, i, path)


def create_directory():
    # Generate directory name based on current timestamp
    timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    directory_name = f"output_{timestamp}"
    # os.makedirs(directory_name)
    # set a directory to save DALL·E images to
    image_dir_name = "dalle_images"
    image_dir = os.path.join(os.curdir, image_dir_name, directory_name)

    # create the directory if it doesn't yet exist
    if not os.path.isdir(image_dir):
        os.mkdir(image_dir)

    # print the directory to save to
    print(f"{image_dir=}")
    return image_dir


# create an image
def generate_dalle_img(prompt):
    # call the OpenAI API
    generation_response = client.images.generate(
            model="dall-e-2",
            prompt=prompt,
            n=1,
            response_format="url",
    )
    # print response
    print(generation_response)
    return generation_response


def single_prompt(prompt):
    path = create_directory()
    response = generate_dalle_img(prompt)
    save_image(response, 0, path)


def save_image(generation_response, index, image_dir_name):
    # Generate filename with timestamp
    timestamp = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
    filename = f"{image_dir_name}/image_{index}_{timestamp}.png"

    # Save the image
    # extract image URL from response
    generated_image_url = generation_response.data[
        0].url
    # download the image
    generated_image = requests.get(
            generated_image_url).content
    # write the image to the file
    with open(filename, "wb") as image_file:
        image_file.write(generated_image)
    # print the image
    print(f"Image {index} saved as {filename}")

filename = 'output.txt'  # Replace with your file path
read_file_and_process(filename)

#single_prompt("king of sea sharks")


Playground AI  - Stable diffusion Model

In [ ]:
# imports
import os  # used to access filepaths
from datetime import datetime
import torch
from diffusers import DiffusionPipeline


def read_file_and_process(filename):
    path = create_directory()
    with open(filename, 'r') as file:
        prompts = file.read()

    prompt = prompts.split('\n\n')
    for i, prompt in enumerate(prompt):
        generate_playground_img(prompt, i, path)


def create_directory():
    # Generate directory name based on current timestamp
    timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    directory_name = f"output_{timestamp}"
    # os.makedirs(directory_name)
    # set a directory to save Playground images to
    image_dir_name = "playground_images"
    # create the directory if it doesn't yet exist
    if not os.path.isdir(image_dir_name):
        os.mkdir(image_dir_name)

    image_dir = os.path.join(os.curdir, image_dir_name, directory_name)

    # create the directory if it doesn't yet exist
    if not os.path.isdir(image_dir):
        os.mkdir(image_dir)

    # print the directory to save to
    print(f"{image_dir=}")
    return image_dir


# create an image
def generate_playground_img(prompt,index,path):
    pipe = DiffusionPipeline.from_pretrained(
    "playgroundai/playground-v2.5-1024px-aesthetic",
    torch_dtype=torch.float16,
    variant="fp16",
    ).to("cuda")


    # Generate filename with timestamp

    timestamp = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
    filename = f"{path}/image_{index}_{timestamp}.png"

    image = pipe(prompt=prompt, num_inference_steps=50, guidance_scale=3).images[0]
    image.save(filename)
    print(f"Image {index} saved as {filename}")


def single_prompt(prompt):
    generate_playground_img(prompt, 0)


filename = 'prompts.txt'  # Replace with your file path
read_file_and_process(filename)

#single_prompt("Boy in a boat with tiger in the middle of the ocean")